In [1]:
# IMPORTS
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamgamelog, leaguegamefinder, leaguegamelog, commonteamroster, teamdashlineups, commonplayerinfo
from nba_api.live.nba.endpoints import scoreboard, playbyplay, boxscore
from datetime import datetime, timezone
from dateutil import parser
import pandas as pd
from pprint import pprint

In [2]:
# GET THE AVERAGE STATS (PTS, RBS, AST) ON THE CURRENT SEASON FOR A PLAYER

def calculate_player_averages(target_player_name):
    # Get the list of all NBA players
    nba_players = players.get_players()

    # Find the player with the specified name
    target_player = next(player for player in nba_players if player['full_name'] == target_player_name)

    # Extract the relevant data for the player's career
    career_stats = playercareerstats.PlayerCareerStats(player_id=target_player['id'])
    career_per_season_data = career_stats.get_data_frames()[0]

    # Create a DataFrame for the player's career stats
    player_career_stats_df = pd.DataFrame(career_per_season_data)

    # Create a DataFrame with player information
    nba_players_df = pd.DataFrame(nba_players)

    # Get the player's game log for the current season
    game_log = playergamelog.PlayerGameLog(player_id=target_player['id'], season="2023-24")
    game_log_data = game_log.get_data_frames()[0]

    # Create a DataFrame for the player's game log
    game_log_df = pd.DataFrame(game_log_data)
    game_log_df['PLAYER_NAME'] = target_player_name

    # Calculate average points per game (PPG), average rebounds per game (RPG),
    # average assists per game (APG), and average minutes per game (MPG)
    average_stats = game_log_df.groupby('PLAYER_NAME').agg({
        'PTS': 'mean',
        'REB': 'mean',
        'AST': 'mean',
        'MIN': 'mean'
    }).reset_index()

    return average_stats

# Example usage
target_player_name = "Tyrese Maxey"
average_stats = calculate_player_averages(target_player_name)
average_stats

,PLAYER_NAME,PTS,REB,AST,MIN
0,Tyrese Maxey,25.630435,3.478261,6.347826,37.086957
